<a href="https://colab.research.google.com/github/dweebee/pytorch-practices/blob/main/00_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

'''
    07-2. 심층신경망(p.411~421)
    파이토치
'''

 파이토치로 신겸앙 모델 만들기

 주소: https://bit.ly/hg2-07-2-pt 접속시 코랩에서 이 절의 코드를 바로 열어 볼 수 있습니다.

파이토치의 컴퓨터 비전 라이브러리인 torchvision을 통해 데이터셋을 로드할 수 있습니다.

In [ ]:
from torchvision.datasets import FashionMNIST

fm_train = FashionMNIST(root='.', train=True, download=True)
fm_test = FashionMNIST(root='.', train=False, download=True)

type(fm_train)

100%|██████████| 26.4M/26.4M [00:01<00:00, 15.5MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 231kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 4.29MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 10.2MB/s]


torchvision.datasets.mnist.FashionMNIST

- torchvision: PyTorch에서 자주 쓰이는 이미지 데이터셋과 전처리 도구를 제공
- MNIST처럼 28x28 흑백 이미지지만, 숫자가 아니라 의류(옷) 이미지 데이터셋
    - 클래스: 티셔츠, 바지, 스니커즈, 샌들 등 10가지 분류.
- root='.': 현재 디렉토리에 클래스명과 동일한 FashionMNIST 폴더를 만들고 그 안에 데이터를 저장하겠다는 의미. 다운된 데이터를 저장할 위치.
- download=True: 로컬에 이미 데이터가 있다면 다운로드 하지 않고, 데이터가 없으면 자동으로 다운로드 | 기본값은 False

In [ ]:
type(fm_train.data)

torch.Tensor

tensor는 파이토치의 기본 데이터 구조.

넘파이 배열과 비슷한 인터페이스를 사용.

In [ ]:
print(fm_train.data.shape, fm_test.data.shape)

torch.Size([60000, 28, 28]) torch.Size([10000, 28, 28])


# PyTorch 이미지 텐서의 Shape과 전처리 요약

PyTorch에서 이미지 데이터를 모델에 입력하기 위해서는 일정한 텐서 구조와 전처리 방식이 필요하다.

---

## 기본 텐서 Shape: `(N, C, H, W)`

| 차원 | 의미 |
|------|------|
| `N` | 이미지 수 또는 배치 크기 (`batch size`) |
| `C` | 채널 수 (흑백: 1, RGB: 3) |
| `H` | 이미지 높이 (Height) |
| `W` | 이미지 너비 (Width) |

예시:
```python
x = torch.randn(32, 3, 224, 224)  # 32장의 RGB 이미지
```

---

## 왜 `C`가 두 번째일까?

- PyTorch는 Channel-first 구조 `(C, H, W)`를 따름
- `torch.nn.Conv2d`, `BatchNorm2d` 등 모든 이미지 연산이 이 구조를 전제로 설계됨
- `transforms.ToTensor()`를 사용하면 PIL 이미지를 자동으로 `(C, H, W)`로 변환함

---

## 파인튜닝 시: 사전학습 모델 전처리와 일치해야 함

예: `resnet18(pretrained=True)`는 ImageNet 기준으로 학습되었기 때문에 입력 전처리가 반드시 일치해야 함

```python
transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])
```

| 요구 조건 | 값 |
|-----------|-----|
| 크기      | 224×224 |
| 채널 수   | 3 (RGB) |
| 정규화    | ImageNet 기준 평균과 표준편차

---

## `N`은 이미지 수? 배치 크기?

- 전체 데이터셋: 이미지 수 (`len(dataset)`)
- 모델 입력 시: 배치 크기 (`batch_size=32` 등)

```python
for images, labels in dataloader:
    print(images.shape)  # (32, 3, 224, 224)
```

→ 문맥에 따라 `N`은 이미지 수 또는 배치 크기 둘 다 의미할 수 있다.

---

## 요약

- PyTorch는 `(N, C, H, W)` 형식을 사용한다
- `C`는 Conv 연산 특성상 반드시 두 번째에 위치해야 한다
- 사전학습 모델을 쓸 경우 입력 전처리를 반드시 맞춰야 한다
- `N`은 데이터의 수이자 모델 처리 단위인 배치 크기이다


In [ ]:
print(fm_train.targets.shape, fm_test.targets.shape)

torch.Size([60000]) torch.Size([10000])


In [ ]:
train_input = fm_train.data
train_target = fm_train.targets

In [ ]:
fm_train.data[0]

tensor([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,   0,
           0,  13,  73,   0,   0,   1,   4,   0,   0,   0,   0,   1,   1,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,   0,
          36, 136, 127,  62,  54,   0,   0,   0,   1,   3,   4,   0,   0,   3],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   6,   0,
         102, 204, 176, 134, 144, 123,  23,   0,   0,   0,   0,  12,  10,   0],
        [  0,   0,   0,   0,   0,   0,   0,   

- 값들은 0 ~ 255 사이의 정수형 (uint8) 값
- 실제 이미지 시각화
```
    import matplotlib.pyplot as plt
    plt.imshow(fm_train.data[0], cmap='gray')
    plt.show()


In [ ]:
fm_train.data[0].shape #각 이미지에 대한 정답 레이블 (0~9)

torch.Size([28, 28])

In [ ]:
fm_train.targets[0]

tensor(9)

In [ ]:
fm_train.targets[0].shape

torch.Size([])

- 모양(shape)이 없는 0차원 텐서, 즉 스칼라
- .item()을 쓰면 스칼라 tensor를 Python 숫자로 변환가능

In [ ]:
# 정규화
train_scaled = train_input / 255.0

- 0~1 범위로 정규화하면: 학습이 더 안정적이고 경사하강법의 수렴 속도도 좋아짐

In [ ]:
train_scaled.data[0]

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0039, 0.0000, 0.0000, 0.0510, 0.2863, 0.0000,
         0.0000, 0.0039, 0.0157, 0.0000, 0.0000, 0.0000

In [ ]:
from sklearn.model_selection import train_test_split

train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)
print(train_scaled.shape, val_scaled.shape)

torch.Size([48000, 28, 28]) torch.Size([12000, 28, 28])


In [ ]:
train_scaled[0].shape

torch.Size([28, 28])

- 첫 번째 이미지는 28픽셀 × 28픽셀의 2차원 텐서
- 채널(C)은 따로 없는데, 이유는 흑백(grayscale)이기 때문
- 모델 입력으로 넣기 전에 unsqueeze(0)을 통해 채널 차원을 추가해 (1, 28, 28)으로 만드는 게 일반적
```
img = fm_train.data[0].unsqueeze(0).float() / 255.0  # (1, 28, 28)


In [ ]:
import torch.nn as nn

model = nn.Sequential(
    nn.Flatten(),           # (1) 2D 이미지 → 1D 벡터
    nn.Linear(784, 100),    # (2) 입력층 → 은닉층 | 입력 벡터(784차원)를 은닉층(100차원)으로 매핑하는 완전연결층
    nn.ReLU(),              # (3) 활성화 함수 | 비선형성 추가
    nn.Linear(100, 10)      # (4) 은닉층 → 출력층 | 은닉층 출력을 최종 클래스 10개로 매핑 (소프트맥스 전 단계)
)


Sequential 클래스 안에 필요한 층을 차례로 나열한다.

시작할 때 1차원으로 펴주고, 두 밀집층 사이에 활성화함수 추가.

Linear는 밀집층인데, 호출시에는 입력크기와 출력크기(뉴런개수)를 매개변수로 반드시 전달한다.

파이토치에서는 활성화함수를 별도의 층으로 추가한다.

출력층에 해당하는 두번째 밀집층 다음에 활성화 함수가 없다. 파이토치에서 다중분류시 이를 생략한다.

In [ ]:
!pip install -q torchinfo

In [ ]:
from torchinfo import summary

summary(model)

Layer (type:depth-idx)                   Param #
Sequential                               --
├─Flatten: 1-1                           --
├─Linear: 1-2                            78,500
├─ReLU: 1-3                              --
├─Linear: 1-4                            1,010
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0

파이토치에서 모델 전체 구조 확인시 torchinfo 패키지를 활용한다.

입력 크기를 지정하면 입력 데이터가 각 층을 통과할 때 어떻게 크기가 변하는지 확인할 수 있다.

In [ ]:
summary(model, input_size=(32,28,28))

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [32, 10]                  --
├─Flatten: 1-1                           [32, 784]                 --
├─Linear: 1-2                            [32, 100]                 78,500
├─ReLU: 1-3                              [32, 100]                 --
├─Linear: 1-4                            [32, 10]                  1,010
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 2.54
Input size (MB): 0.10
Forward/backward pass size (MB): 0.03
Params size (MB): 0.32
Estimated Total Size (MB): 0.45

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

cpu


Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=100, bias=True)
  (2): ReLU()
  (3): Linear(in_features=100, out_features=10, bias=True)
)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
epochs = 5
batches = int(len(train_scaled)/32)
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for i in range(batches):
        inputs = train_scaled[i*32:(i+1)*32].to(device)
        targets = train_target[i*32:(i+1)*32].to(device)
        optimizer.zero_grad()
        outputs = model(inputs) # 내부적으로 __call__ → forward()를 통해 네트워크를 순전파 | outputs는 softmax 되지 않은 raw 출력값, 즉 logits
        loss = criterion(outputs, targets) # criterion이 반환하는 값은 배치에 있는 샘플에 대한 손실합이 아니라 '평균'
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss/batches:.4f}")

    model.eval()
    with torch.no_grad():
        val_scaled = val_scaled.to(device)
        val_target = val_target.to(device)
        outputs = model(val_scaled)
        predicts = torch.argmax(outputs, dim=1)
        corrects = (predicts == val_target).sum().item()
    acc = corrects / len(val_target)
    print(f"검증 정확도: {acc:.4f}")


Epoch 1/5, Train Loss: 2.3250
검증 정확도: 0.0459
Epoch 2/5, Train Loss: 2.3250
검증 정확도: 0.0459
Epoch 3/5, Train Loss: 2.3250
검증 정확도: 0.0459
Epoch 4/5, Train Loss: 2.3250
검증 정확도: 0.0459
Epoch 5/5, Train Loss: 2.3250
검증 정확도: 0.0459


logits란?
logits는 소프트맥스 함수에 들어가기 전의 값

어떤 클래스에 속할 “가능성”을 상대적인 점수로 표현한 것

보통 [-∞, +∞] 범위의 실수값

정규화된 확률은 아니지만, 순서/크기 관계는 중요